### 1. Scrape weather condition data from website (Link:

`https://www.accuweather.com/en/in/india-weather`

### 2. Create database and save data into it (Use MySQL)
### 3. Make a weather dashboard using Power BI.

In [1]:
# !pip install beautifulsoup4 fake_headers webdriver-manager lxml

In [2]:
# !pip install selenium

In [3]:
# Importing the Libraries
from bs4 import BeautifulSoup
from fake_headers import Headers
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [4]:
keyword = "India"

In [5]:
# Website Link
url = f'https://www.accuweather.com/en/search-locations?query={keyword}'

In [6]:
options = Options()
options.headless = True

header = Headers(
        browser="chrome",  # Generate only Chrome UA
        headers=False,  # generate misc headers
)
custom_user_agent = header.generate()['User-Agent']
options.add_argument(f"user-agent={custom_user_agent}")


C:\Users\DEV-Study\AppData\Local\Temp\ipykernel_12140\1560318565.py:2: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [7]:
driver_path = ChromeDriverManager().install()
driver = webdriver.Chrome(executable_path=driver_path, options=options)
driver.get(url)

[WDM] - Downloading: 100%|█████████████████████████████████████████████████████████| 6.79M/6.79M [00:16<00:00, 429kB/s]
C:\Users\DEV-Study\AppData\Local\Temp\ipykernel_12140\805700613.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path, options=options)


In [8]:
soup = BeautifulSoup(driver.page_source, 'lxml')
driver.close()

In [9]:
try:
    tag_a_list = soup.find('div', attrs={'class': 'locations-list content-module'}).find_all('a', href=True)
except AttributeError:
    print(f'\n No data found {keyword}')
    tag_a_list = None

In [10]:
count_list = 0
if tag_a_list is not None:
    for a in tag_a_list:
            count_list += 1
            href = a['href']
            city = a.getText(strip=True)
            print(f'{count_list}. {city} => https://www.accuweather.com{href}')

1. India, Mississippi, US => https://www.accuweather.com/web-api/three-day-redirect?key=2196366&target=
2. India, Pennsylvania, US => https://www.accuweather.com/web-api/three-day-redirect?key=2242330&target=
3. India, Tennessee, US => https://www.accuweather.com/web-api/three-day-redirect?key=2086021&target=
4. India, Texas, US => https://www.accuweather.com/web-api/three-day-redirect?key=2145269&target=
5. India, North Bank, GM => https://www.accuweather.com/web-api/three-day-redirect?key=1152086&target=
6. India, Anenii Noi, MD => https://www.accuweather.com/web-api/three-day-redirect?key=1704129&target=
7. India, Zambézia, MZ => https://www.accuweather.com/web-api/three-day-redirect?key=1489646&target=
8. India, La Paz, SV => https://www.accuweather.com/web-api/three-day-redirect?key=1164111&target=
9. India, Amuria, UG => https://www.accuweather.com/web-api/three-day-redirect?key=1-1041285_1_al&target=
10. Indianapolis, Indiana, US => https://www.accuweather.com/web-api/three-day-

In [11]:
import calendar

from bs4 import BeautifulSoup
from fake_headers import Headers
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [12]:
def get_soup(url):
    options = Options()
    options.headless = True

    header = Headers(
        browser="chrome",  # Generate only Chrome UA
        headers=False,  # generate misc headers
    )
    custom_user_agent = header.generate()['User-Agent']
    options.add_argument(f"user-agent={custom_user_agent}")

    driver_path = ChromeDriverManager().install()
    if driver_path is None:
        driver_path = ChromeDriverManager().install()
    driver = webdriver.Chrome(executable_path=driver_path, options=options)
    driver.get(f'https://www.accuweather.com{url}')

    soup = BeautifulSoup(driver.page_source, 'lxml')
    driver.close()

    return soup

In [13]:
def get_detail(selected_city, url_detail):
    soup = get_soup(url_detail)

    current_weather_degree = soup.find('div', attrs={'class': 'cur-con-weather-card__panel'}) \
        .find('div', attrs={'class': 'forecast-container'}) \
        .find('div', attrs={'class': 'temp-container'}) \
        .find('div', attrs={'class': 'temp'}) \
        .getText(strip=True)
    try:
        current_weather_phrase = soup \
            .find('span', attrs={'class': 'phrase'}) \
            .getText(strip=True)
        current_weather_phrase = f', {current_weather_phrase}'
    except AttributeError:
        current_weather_phrase = ''
    try:
        current_weather_minute_cast = soup.find('p', attrs={'class': 'minutecast-banner__phrase'}) \
            .getText(strip=True)
        current_weather_minute_cast = f', {current_weather_minute_cast}'
    except AttributeError:
        current_weather_minute_cast = ''
    current_weather = f'{current_weather_degree}{current_weather_phrase}{current_weather_minute_cast}'

    current_date = soup.find('p', attrs={'class': 'date'}).getText(strip=True)
    current_day = current_date.split('/')[1]
    current_month = current_date.split('/')[0]
    current_date = f'{calendar.month_name[int(current_month)]}, {current_day}'

    current_time = soup.find('p', attrs={'class': 'cur-con-weather-card__subtitle'}).getText(strip=True)

    weather_quality = soup.find('div', attrs={'class': 'cur-con-weather-card__panel details-container'}) \
        .find_all('div', attrs={'class': 'spaced-content detail'})

    air_quality = weather_quality[0].find('span', attrs={'class': 'value'}).getText(strip=True)

    wind = weather_quality[1].find('span', attrs={'class': 'value'}).getText(strip=True)

    wind_gusts = weather_quality[2].find('span', attrs={'class': 'value'}).getText(strip=True)

    tomorrow_weather_card = soup.find('div', attrs={'data-qa': 'tomorrowWeatherCard'}) \
        .find('a') \
        .find('div', attrs={'class': 'card-content'})
    tomorrow_weather_degree = tomorrow_weather_card \
        .find('div', attrs={'class': 'forecast-container'}) \
        .find('div', attrs={'class': 'temp-container'}) \
        .find('div', attrs={'class': 'temp'}) \
        .getText(strip=True)
    tomorrow_weather_phrase = tomorrow_weather_card \
        .find('div', attrs={'class': 'phrase'}) \
        .getText(strip=True)
    tomorrow_weather = f'{tomorrow_weather_degree}, {tomorrow_weather_phrase}'

    weather = {
        'current_weather': current_weather,
        'current_date': current_date,
        'current_time': current_time,
        'air_quality': air_quality,
        'wind': wind,
        'wind_gusts': wind_gusts,
        'tomorrow_weather': tomorrow_weather
    }

    print(f"\n░░░ {selected_city} >>> Weather = {weather} ░░░")
    
    df = pd.DataFrame(weather.items()).T
    df.columns = df.iloc[0]
    df = df.iloc[1:]
    return df

df2 = get_detail('India Gate, Delhi, IN', '/web-api/three-day-redirect?key=328328&target=')

C:\Users\DEV-Study\AppData\Local\Temp\ipykernel_12140\3937221783.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True
C:\Users\DEV-Study\AppData\Local\Temp\ipykernel_12140\3937221783.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path, options=options)



░░░ India Gate, Delhi, IN >>> Weather = {'current_weather': '10°C, Mostly cloudy, Rain starting in 6 min', 'current_date': 'March, 23', 'current_time': '6:38 AM', 'air_quality': '6°', 'wind': 'Fair', 'wind_gusts': 'SW 20 km/h', 'tomorrow_weather': '13°/ 7°, A couple of showers'} ░░░


In [14]:
!pip install mysql-connector-python

In [15]:
import mysql.connector as connection
mydb=connection.connect(host="localhost", user="root", passwd="Manish@12345",database="task", use_pure=True)
cursor=mydb.cursor() #create a cursor to exicute queries

In [16]:
col = df2.columns
col

Index(['current_weather', 'current_date', 'current_time', 'air_quality',
       'wind', 'wind_gusts', 'tomorrow_weather'],
      dtype='object', name=0)

In [17]:
# cursor.execute("""create table task1(
# current_weather varchar(100), 
# current_date_ varchar(100), 
# current_time_ varchar(100),
# air_quality varchar(100),
# wind varchar(100),
# wind_gusts varchar(100),
# tomorrow_weather varchar(100));""")
# # print(cursor.fetchall())

In [18]:
for i in range(len(df2)):
    data = tuple(df2.iloc[i])
    print(data)
    q = f"insert into task1(current_weather, current_date_ , current_time_ ,air_quality ,wind, wind_gusts, tomorrow_weather) VALUES {data}"
    cursor.execute(q)
    

('10°C, Mostly cloudy, Rain starting in 6 min', 'March, 23', '6:38 AM', '6°', 'Fair', 'SW 20 km/h', '13°/ 7°, A couple of showers')


In [19]:
mydb.commit()